# Maps of RMSSSH

In [1]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='make_profiles',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 
cluster.scale(196)
cluster

from dask.distributed import Client
client = Client(cluster)
client



/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


Client Scheduler: tcp://172.30.100.1:39432 Dashboard: http://172.30.100.1:38302/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [2]:
!squeue -u albert7a

import time
nb_workers = 0
while True:
    nb_workers = len(client.scheduler_info()["workers"])
    if nb_workers >= 2:
        break
    time.sleep(1)
print(nb_workers)

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           8190952     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8190953     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8190954     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8190955     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8190956     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8190957     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8190958     hsw24 make_pro albert7a PD       0:00      1 (Priority)
96


## 0. Modules and libraries

In [3]:
import warnings
warnings.filterwarnings("ignore")
import dask 
import numpy as np
import xarray as xr
import time
import numpy.ma as ma
import matplotlib.cm as mplcm
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
import os 
import time 
import glob

import zarr

%matplotlib inline


## 1. Data 

### eNATL60

In [4]:
ds=xr.open_zarr('/store/albert7a/eNATL60/zarr/eNATL60-BLBT02-SSH-1h')
ds_sorted=ds.sortby('time_counter')


In [5]:
ssh_eNATL60=ds_sorted.sossheig

In [9]:
ssh_eNATL60

<xarray.DataArray 'sossheig' (time_counter: 11688, y: 4729, x: 8354)>
dask.array<shape=(11688, 4729, 8354), dtype=float32, chunksize=(24, 240, 480)>
Coordinates:
    time_centered  (time_counter) datetime64[ns] dask.array<shape=(11688,), chunksize=(24,)>
  * time_counter   (time_counter) datetime64[ns] 2009-06-30T00:30:00 ... 2010-10-29T23:30:00
Dimensions without coordinates: y, x
Attributes:
    interval_write:    1 h
    long_name:         sea surface height
    online_operation:  average
    standard_name:     sea_surface_height_above_geoid
    units:             m

In [6]:
ssh_eNATL60_1d_2009=ssh_eNATL60.sel(time_counter=slice('2009-06-30','2009-12-31')).groupby('time_counter.dayofyear').mean(dim='time_counter')
ssh_eNATL60_1d_2010=ssh_eNATL60.sel(time_counter=slice('2010-01-01','2010-10-29')).groupby('time_counter.dayofyear').mean(dim='time_counter')


In [7]:
ssh_eNATL60_1d=xr.concat([ssh_eNATL60_1d_2009,ssh_eNATL60_1d_2010],dim='dayofyear')

In [8]:
std_ssh_eNATL60=ssh_eNATL60_1d.std(dim='dayofyear')

In [29]:
std_ssh_eNATL60

<xarray.DataArray 'sossheig' (y: 4729, x: 8354)>
dask.array<shape=(4729, 8354), dtype=float32, chunksize=(240, 480)>
Dimensions without coordinates: y, x

In [9]:
mean_ssh_eNATL60=ssh_eNATL60_1d.mean(dim='dayofyear')-ssh_eNATL60.mean()

In [31]:
mean_ssh_eNATL60

<xarray.DataArray 'sossheig' (y: 4729, x: 8354)>
dask.array<shape=(4729, 8354), dtype=float32, chunksize=(240, 480)>
Dimensions without coordinates: y, x

In [32]:
ssh_eNATL60.attrs

OrderedDict([('interval_write', '1 h'),
             ('long_name', 'sea surface height'),
             ('online_operation', 'average'),
             ('standard_name', 'sea_surface_height_above_geoid'),
             ('units', 'm')])

In [33]:
std_dataset=std_ssh_eNATL60.to_dataset(name='std_sossheig')
std_dataset['std_sossheig'].attrs=ssh_eNATL60.attrs
std_dataset['std_sossheig'].attrs['standard_name']='standart deviation of ssh'
std_dataset['std_sossheig'].attrs['long_name']='standart deviation of sea surface height'
std_dataset.attrs['global_attribute']= 'standart deviation of ssh computed on occigen '
std_dataset.to_netcdf(path='/scratch/cnt0024/hmg2840/albert7a/eNATL60/eNATL60-BLBT02-S/1h/SSH-MEAN-STD/eNATL60-BLBT02_std-ssh_1d.nc',mode='w')

distributed.utils - ERROR - deque index out of range
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/scheduler.py", line 715, in update
    istart=self.index, workers=self.workers, start_boundary=boundary
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/diagnostics/task_stream.py", line 79, in rectangles
    msg = self.buffer[i]
IndexError: deque index out of range
distributed.utils - ERROR - deque index out of range
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/scheduler.py", line 715, in update
    istar

distributed.utils - ERROR - deque index out of range
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/scheduler.py", line 715, in update
    istart=self.index, workers=self.workers, start_boundary=boundary
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/diagnostics/task_stream.py", line 79, in rectangles
    msg = self.buffer[i]
IndexError: deque index out of range
distributed.utils - ERROR - deque index out of range
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/scheduler.py", line 715, in update
    istar

distributed.utils - ERROR - deque index out of range
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/scheduler.py", line 715, in update
    istart=self.index, workers=self.workers, start_boundary=boundary
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/diagnostics/task_stream.py", line 79, in rectangles
    msg = self.buffer[i]
IndexError: deque index out of range
distributed.utils - ERROR - deque index out of range
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/scheduler.py", line 715, in update
    istar

distributed.utils - ERROR - deque index out of range
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/scheduler.py", line 715, in update
    istart=self.index, workers=self.workers, start_boundary=boundary
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/diagnostics/task_stream.py", line 79, in rectangles
    msg = self.buffer[i]
IndexError: deque index out of range
distributed.utils - ERROR - deque index out of range
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/scheduler.py", line 715, in update
    istar

distributed.utils - ERROR - deque index out of range
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/scheduler.py", line 715, in update
    istart=self.index, workers=self.workers, start_boundary=boundary
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/diagnostics/task_stream.py", line 79, in rectangles
    msg = self.buffer[i]
IndexError: deque index out of range
distributed.utils - ERROR - deque index out of range
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/scheduler.py", line 715, in update
    istar

...

distributed.utils - ERROR - deque index out of range
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/scheduler.py", line 715, in update
    istart=self.index, workers=self.workers, start_boundary=boundary
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/diagnostics/task_stream.py", line 79, in rectangles
    msg = self.buffer[i]
IndexError: deque index out of range
distributed.utils - ERROR - deque index out of range
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/scheduler.py", line 715, in update
    istar

distributed.utils - ERROR - deque index out of range
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/scheduler.py", line 715, in update
    istart=self.index, workers=self.workers, start_boundary=boundary
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/diagnostics/task_stream.py", line 79, in rectangles
    msg = self.buffer[i]
IndexError: deque index out of range
distributed.utils - ERROR - deque index out of range
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/scheduler.py", line 715, in update
    istar

distributed.utils - ERROR - deque index out of range
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/scheduler.py", line 715, in update
    istart=self.index, workers=self.workers, start_boundary=boundary
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/diagnostics/task_stream.py", line 79, in rectangles
    msg = self.buffer[i]
IndexError: deque index out of range
distributed.utils - ERROR - deque index out of range
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/scheduler.py", line 715, in update
    istar

distributed.utils - ERROR - deque index out of range
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/scheduler.py", line 715, in update
    istart=self.index, workers=self.workers, start_boundary=boundary
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/diagnostics/task_stream.py", line 79, in rectangles
    msg = self.buffer[i]
IndexError: deque index out of range
distributed.utils - ERROR - deque index out of range
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/scheduler.py", line 715, in update
    istar

distributed.utils - ERROR - deque index out of range
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/scheduler.py", line 715, in update
    istart=self.index, workers=self.workers, start_boundary=boundary
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/diagnostics/task_stream.py", line 79, in rectangles
    msg = self.buffer[i]
IndexError: deque index out of range
distributed.utils - ERROR - deque index out of range
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/scheduler.py", line 715, in update
    istar

In [10]:
mean_ssh_eNATL60=mean_ssh_eNATL60.to_dataset(name='mean_sossheig')
mean_ssh_eNATL60['mean_sossheig'].attrs=ssh_eNATL60.attrs
mean_ssh_eNATL60['mean_sossheig'].attrs['standard_name']='temporal mean of ssh'
mean_ssh_eNATL60['mean_sossheig'].attrs['long_name']='temporal mean of sea surface height'
mean_ssh_eNATL60.attrs['global_attribute']= 'temporal mean of ssh computed on occigen '
mean_ssh_eNATL60.to_netcdf(path='/scratch/cnt0024/hmg2840/albert7a/eNATL60/eNATL60-BLBT02-S/1h/SSH-MEAN-STD/eNATL60-BLBT02_mean-ssh_1d.nc',mode='w')

distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took